## Assemble website data

This notebook assembles the data for the website.


This is the info to Liz on 11/16/18:


Here's the data for the website:
```
kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='studies'
    )
)

kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='recordings'
    )
)

kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='true_units'
    )
)

kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='sorters'
    )
)
```


In [0]:
# Only run this cell if you are running this on a hosted runtime that does not have these packages installed
# Consider connecting to a local runtime
%%capture
!pip install spikeforest

In [0]:
from kbucket import client as kb
import spikeforest as sf
import spikeextractors as si
import json
import numpy as np

In [0]:
## Configure read/write access to kbucket
sf.kbucketConfigRemote(name='spikeforest1-readwrite',ask_password=True)

In [0]:
recording_collection_names=[
    'spikeforest_magland_synth_recordings',
    'spikeforest_bionet8c_recordings',
    'spikeforest_bionet32c_recordings',
    'spikeforest_mearec_tetrode_recordings'
]

batch_names=[
    'summarize_recordings_magland_synth','summarize_recordings_bionet8c','summarize_recordings_bionet32c','summarize_recordings_mearec_tetrode',
    'ms4_magland_synth','irc_magland_synth','sc_magland_synth',
    'ms4_bionet8c','irc_bionet8c','sc_bionet8c',
    'ms4_bionet32c','irc_bionet32c','sc_bionet32c',
    'ms4_mearec_tetrode','irc_mearec_tetrode','sc_mearec_tetrode'
]

In [0]:
## Load the SpikeForest data
SF=sf.SFData()
for rcname in recording_collection_names:
  print(rcname)
  SF.loadRecordings(key=dict(name=rcname))
for bname in batch_names:
  print(bname)
  SF.loadProcessingBatch(batch_name=bname)

In [0]:
def min_max_range(a):
  return [min(a),max(a)]

def compute_recording_ranges(study):
  rnames=study.recordingNames()
  summary_objects=[study.recording(rname).getSummaryObject() for rname in rnames]
  recording_dirnames=[study.recording(rname).directory() for rname in rnames]
  
  duration_sec_list=[float(obj['computed_info']['duration_sec']) for obj in summary_objects]
  num_channels_list=[int(obj['computed_info']['num_channels']) for obj in summary_objects]
  samplerate_hz_list=[int(obj['computed_info']['samplerate']) for obj in summary_objects]
  num_true_units_list=[int(len(study.recording(rname).trueUnitsInfo(format='json'))) for rname in rnames]
  file_size_bytes_list=[int(kb.getFileSize(dirname+'/raw.mda')) for dirname in recording_dirnames]
  
  recording_ranges=dict(
      duration_sec=min_max_range(duration_sec_list),
      num_channels=min_max_range(num_channels_list),
      samplerate_hz=min_max_range(samplerate_hz_list),
      file_size_bytes=min_max_range(file_size_bytes_list),
      num_ground_truth_units=min_max_range(num_true_units_list)
  )
  
  return recording_ranges

In [0]:
## Load the studies
print('Loading studies')
all_studies=[]
studies_by_name=dict()
for sname in SF.studyNames():
  print(sname)
  study=dict(name=sname)
  study['sorters']=[] # initialize
  study['num_recordings']=len(SF.study(sname).recordingNames())
  study['recording_ranges']=compute_recording_ranges(SF.study(sname))
  studies_by_name[sname]=study
  all_studies.append(study)

In [0]:
## Load the recordings
print('Loading recordings')
all_recordings=[]
for sname in SF.studyNames():
  SS=SF.study(sname)
  for rname in SS.recordingNames():
    RR=SS.recording(rname)
    recording=RR.getObject()
    all_recordings.append(recording)

In [0]:
## Load the units
all_true_units=[]
unit_lookup=dict()
for sname in SF.studyNames():
  SS=SF.study(sname)
  for rname in SS.recordingNames():
    RR=SS.recording(rname)
    true_units_info=RR.trueUnitsInfo(format='json')
    for unit in true_units_info:
      unit['study']=sname
      unit['recording']=rname
      unit['sorting_results']=dict()
      all_true_units.append(unit)
      code=sname+'---'+rname+'---'+str(unit['unit_id'])
      unit_lookup[code]=unit
print('Found {} true units'.format(len(all_true_units)))

In [0]:
## Load the sorting results
print('Loading sorting results')
count=0
sorters_by_name=dict()
for sname in SF.studyNames():
  SS=SF.study(sname)
  for rname in SS.recordingNames():
    RR=SS.recording(rname)
    for srname in RR.sortingResultNames():
      SR=RR.sortingResult(srname)
      result=SR.getObject()
      if not SR.sorterName() in studies_by_name[sname]['sorters']:
        studies_by_name[sname]['sorters'].append(SR.sorterName())
      obj=SR.comparisonWithTruth(format='json')
      sorter=dict(
          name=result['sorter_name'],
          processor_name=result['sorting_processor_name'],
          processor_version=result['sorting_processor_version'],
          params=result['sorting_params']
      )
      sorters_by_name[SR.sorterName()]=sorter
      for ii in obj:
        unit=obj[ii]
        unit_id=unit['unit_id']
        code=sname+'---'+rname+'---'+str(unit_id)
        if code in unit_lookup:
          unit_lookup[code]['sorting_results'][SR.sorterName()]=unit
          count=count+1
print('Loaded {} sorted units'.format(count))


In [0]:
target='spikeforest_website_dev_12_18_2018'

all_sorters=[]
for sname in sorters_by_name:
  all_sorters.append(sorters_by_name[sname])
print('Found {} sorters'.format(len(all_sorters)))

print('Saving {} studies'.format(len(all_studies)))
kb.saveObject(
    key=dict(
        target=target,
        name='studies'
    ),
    object=dict(
        studies=all_studies
    )
)
    
print('Saving {} recordings'.format(len(all_recordings)))
kb.saveObject(
    key=dict(
        target=target,
        name='recordings'
    ),
    object=dict(
        recordings=all_recordings
    )
)


print('Saving {} true units'.format(len(all_true_units)))
kb.saveObject(
    key=dict(
        target=target,
        name='true_units'
    ),
    object=dict(
        true_units=all_true_units
    )
)

print('Saving {} sorters'.format(len(all_sorters)))
kb.saveObject(
    key=dict(
        target=target,
        name='sorters'
    ),
    object=dict(
        sorters=all_sorters
    )
)

In [0]:
print('Study:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='studies'
    )
)
print(json.dumps(obj['studies'][0],indent=4))

print('Recording:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='recordings'
    )
)
print(json.dumps(obj['recordings'][0],indent=4))


print('Unit:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='true_units'
    )
)
print(json.dumps(obj['true_units'][0],indent=4))

print('Sorter:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='sorters'
    )
)
print(json.dumps(obj['sorters'][0],indent=4))

In [0]:
print('studies: '+kb.findFile(key=dict(
    target='spikeforest_website_dev',
    name='studies'
),local=False,remote=True))

print('recordings: '+kb.findFile(key=dict(
    target='spikeforest_website_dev',
    name='recordings'
),local=False,remote=True))

print('units: '+kb.findFile(key=dict(
    target='spikeforest_website_dev',
    name='true_units'
),local=False,remote=True))

print('sorters: '+kb.findFile(key=dict(
    target='spikeforest_website_dev',
    name='sorters'
),local=False,remote=True))